In [ ]:
import mqr
from mqr.plot import Figure
from mqr.nbtools import hstack, vstack, grab_figure

# Datasets

In [ ]:
import pandas as pd

data = pd.read_csv(mqr.sample_data('study-random-5x5.csv'), index_col='run')

# Process

In [ ]:
from mqr.process import Sample, Summary, Specification

In [ ]:
# Just a sample
sample = Sample(data['KPO1'])

hyp_mean = 160.00

with Figure(4, 4, 3, 1, height_ratios=(4, 1, 1)) as (fig, axs):
    mqr.plot.process.summary(sample, axs, hyp_mean=hyp_mean)
    plot = grab_figure(fig)

hstack(
    plot,
    sample,
    vstack(
        sample.conf_mean,
        sample.conf_std,
        mqr.inference.mean.test_1sample(sample.data, hyp_mean),
    ),
)

In [ ]:
summary = Summary(data.loc[:, 'KPI1':'KPO2'])
summary

In [ ]:
import numpy as np
import pickle

# Use the mean and covariance from the random generator
# to construct some good and bad processes
with open(mqr.sample_data('study-random-5x5-mean-cov.pkl'), 'rb') as f:
    mean, cov = pickle.load(f)

offset2 = 2*np.sqrt(cov[2, 2])

---
The `Process` type is a model of a process with various input and output quantities. It calculates metrics like capability and expected defect rate.

In [ ]:
import numpy as np

specs = {
    'KPI1': Specification(mean[0], mean[0]-np.sqrt(cov[0,0])*6, mean[0]+np.sqrt(cov[0,0])*6),
    'KPI2': Specification(mean[1], mean[1]-np.sqrt(cov[1,1])*6, mean[1]+np.sqrt(cov[1,1])*6),
    'KPI3': Specification(mean[2]+offset2, mean[2]-np.sqrt(cov[2,2])*6+offset2, mean[2]+np.sqrt(cov[2,2])*6+offset2),
    'KPO1': Specification(mean[3], mean[3]-np.sqrt(cov[3,3])*6, mean[3]+np.sqrt(cov[3,3])*6),
    'KPO2': Specification(mean[4], mean[4]-np.sqrt(cov[4,4])*2, mean[4]+np.sqrt(cov[4,4])*2),
}
summary = Summary(data.loc[:, 'KPI1':'KPO2'], specs)
summary.capabilities

---
The main process overlays:
* short- and long-term fitted Gaussians, and 
* specification limits.

In [ ]:
with Figure(6, 4, 2, 1) as (fig, ax):
    mqr.plot.process.pdf(summary['KPI1'], show_long_term=True, ax=ax[0])
    mqr.plot.process.tolerance(specs['KPI1'], ax=ax[1])
    ax[0].legend()
    ax[1].legend()

---
The overlays on top of histograms

In [ ]:
with Figure(7, 6, 3, 1) as (fig, ax):
    mqr.plot.process.capability(summary, 'KPI1', cp=6/3, show_long_term=True, ax=ax[0])
    mqr.plot.process.capability(summary, 'KPI3', cp=5/3, show_long_term=True, ax=ax[1])
    mqr.plot.process.capability(summary, 'KPO2', cp=3/3, show_long_term=False, ax=ax[2])
    ax[0].legend()
    ax[1].legend()
    ax[2].legend()

---
Correlations between all KPIs and KPOs

In [ ]:
with Figure(7, 7, 5, 5) as (fig, ax):
    mqr.plot.correlation.matrix(
        data.loc[:, 'KPI1':'KPO2'],
        show_conf=True,
        conf=0.95,
        ax=ax, fig=fig)

---
# Scatter plot

In [ ]:
with Figure(3, 3) as (fig, ax):
    ax.scatter(data['KPI2'], data['KPO2'])

---
# Probability plots

In [ ]:
import scipy.stats as st
import probscale

with Figure(3, 3) as (fig, ax):
    probscale.probplot(
        data['KPI1'],
        dist=st.norm,
        bestfit=True,
        probax='y',
        ax=ax)

The library `reliability` (for reliability modelling) is excellent. Unfortunately, its plotting interface doesn't allow plotting into axes that have already been created (for example with `matplitlib.pyplot.subplots` or `mqr.plots.Figure`). Instead, `reliability` returns the figure, then it can be manipulated.

In [ ]:
import probscale

with Figure(5, 4) as (fig, ax):
    probscale.probplot(
        data['KPO1'],
        probax='y',
        bestfit=True,
        estimate_ci=True,
    )
    ax.grid()
    ax.set_xlabel('KPI1')
    ax.set_ylabel('Fraction below')
    ax.set_yticks([1, 5, 10, 20, 35, 50, 65, 80, 90, 95, 99])
    plot = mqr.nbtools.grab_figure(fig)

hstack(plot, data.head())